In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
from dash import ctx
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Imports the CRUD Python Module
from CS340CRUDModule import AnimalShelter


###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "hardpassword"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__) 

# Loads and encrypts the Grazios Logo
image_filename = 'GraziosoSalvareLogo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


# Dashboard Layout
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(
        children=[
            html.A(
                # Displays the image at the top of the screen centered
                html.Img(
                    src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                    style={'width': '12%', 'height': '12%', 'margin-left': 'auto', 'margin-right': 'auto'}),
                # Adds a URL anchor tag to the logo
                href='https://www.snhu.edu')],
        style={'text-align': 'center'}
    ),
    # Title of the dashboard
    html.Center(html.B(html.H1('Rescue Animal Dashboard',
                       style={'textAlign':'center', 'color': '#ffffff', 'backgroundColor': '#990033', 
                              'border': '3px solid gray', 'fontSize': '42px', 'textShadow': '2px 2px 4px #000000'}))),
    html.Hr(),
    # Interactive buttons that filter data within the db
    html.Div(className='buttonRow',
            style={'display': 'flex'},
                children=[
                    html.Button(id='submit-button-one', n_clicks=0, children='Water Rescue'),
                    html.Button(id='submit-button-two', n_clicks=0, children='Mountain or Wilderness Rescue'),
                    html.Button(id='submit-button-three', n_clicks=0, children='Disaster Rescue or Individual Tracking'),
                    html.Button(id='submit-button-four', n_clicks=0, children='Reset')
                ]),
    html.Hr(),
    # Show Data Table 
    dash_table.DataTable(id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
        style_header={'fontWeight': 'bold', 'border': '1px solid black', 'backgroundColor': '#bfbfbf', 
                      'color': 'black'},
        style_cell={'textAlign': 'left', 'border': '1px solid grey', 'fontWeight': 'normal'},
        ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.Hr(),
    # Displays a unique developer identifier at the bottom of the page
    html.Footer('Web Page Developed By: Jeffery Pierce Waren', 
                    style={'textAlign':'center', 'color': '#ffffff', 'backgroundColor': '#990033', 
                           'border': '3px solid gray', 'fontSize': '18px', 'textShadow': '1px 1px 3px #000000'})
])


#############################################
# Interaction Between Components / Controller
#############################################

#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback responds to the 4 filter buttons and updates the data on the web page dynamically
@app.callback(Output('datatable-id','data'),
              [Input('submit-button-one', 'n_clicks'),
               Input('submit-button-two', 'n_clicks'),
               Input('submit-button-three', 'n_clicks'),
               Input('submit-button-four', 'n_clicks')
              ])
def update_dashboard(button1, button2, button3, button4): 
    # start case   
    df = pd.DataFrame.from_records(db.read({}))
    if "submit-button-one" == ctx.triggered_id:
        water_breeds = ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']
        water_query = {
            "animal_type": "Dog",
            "breed": {"$in": water_breeds},
            "sex_upon_outcome": "Intact Female", 
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
        df = pd.DataFrame.from_records(db.read(water_query))
    elif "submit-button-two" == ctx.triggered_id:
        mountain_breeds = ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']        
        mountain_query = {
            "animal_type": "Dog",
            "breed": {"$in": mountain_breeds},
            "sex_upon_outcome": "Intact Male", 
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }        
        df = pd.DataFrame.from_records(db.read(mountain_query))
    elif "submit-button-three" == ctx.triggered_id:
        disaster_breeds = ['Doberman Pinsch', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']
        disaster_query = {
            "animal_type": "Dog",
            "breed": {"$in": disaster_breeds},
            "sex_upon_outcome": "Intact Male", 
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }        
        df = pd.DataFrame.from_records(db.read(disaster_query))        
    elif "submit-button-four" == ctx.triggered_id:
        df = pd.DataFrame.from_records(db.read({}))      
    # Cleanup Mongo _id field
    df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records')


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ##FIX ME ####
    #add code for chart of your choice (e.g. pie chart) 
    df = pd.DataFrame(viewData)
    breed_counts = df['breed'].value_counts()
    # Ensures the pie chart only displays the top 6 most prevalent breeds
    top_breeds = breed_counts.head(6)
    filtered_df = df[df['breed'].isin(top_breeds.index)]
    return [
        dcc.Graph(            
           figure = px.pie(filtered_df, names='breed', title='Preferred Rescue Animals', 
                    color_discrete_sequence=px.colors.sequential.RdBu, hole=.25)
                    .update_layout(title_font=dict(size=30, color='black'), 
                                   height=620, width=620,
                                   legend=dict(font=dict(size=14, color='black')), 
                                   font=dict(size=14))
       )    
    ]
    


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Saves the lat and long to ensure they are centered within the map    
    latitude = dff.iloc[row, 13]
    longitude = dff.iloc[row, 14]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=8, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[latitude, longitude], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


app.run_server(debug=True)

Connection Successful
Dash app running on http://127.0.0.1:19001/
